In [5]:
#! pip install streamlit-option-menu
#! pip install prophet


In [ ]:
import yfinance as yf
import streamlit as st
import time
import pandas as pd
from streamlit_option_menu import option_menu
import numpy as np
from datetime import date,datetime,timedelta
from plotly import graph_objs as go
from prophet import Prophet


st.set_page_config(page_title="Stock Analysis", page_icon=":chart_with_upwards_trend:")

st.title("Stocks Analysis")
st.caption("Stock Price Analysis and Price forecasting of National Stock Exchange (NSE) and Bombay Stock Exchange (BSE) listed Companies.")
st.write("---")

with st.sidebar:
    selected=option_menu(
    menu_title="Menu",
    options=["HOME","LIVE PRICES","GRAPHS","DATA","INFORMATION","TECHNICAL INDICATORS","FORECAST","HELP"],
    icons=["house","activity","graph-up","clipboard-data","info-circle","clipboard-pulse","bar-chart-line","question-circle"],menu_icon="shop-window")
    st.write("---")
    

company = ("AAPL")


nifty = yf.Ticker('^NSEI')
currentN = nifty.history(period='5d')['Close'].iloc[-1]
lastN = nifty.history(period='5d')['Close'].iloc[-2]
changeN = currentN - lastN
percentage_changeN = (changeN / lastN) * 100
current_formattedN = f"₹{currentN:.2f}"
change_formattedN = f"{changeN:.2f}"
percentage_change_formattedN = f"{percentage_changeN:.2f}%"

sensex = yf.Ticker('^BSESN')
currentS = sensex.history(period='5d')['Close'].iloc[-1]
lastS = sensex.history(period='5d')['Close'].iloc[-2]
changeS = currentS - lastS
percentage_changeS = (changeS / lastS) * 100
current_formattedS = f"₹{currentS:.2f}"
change_formattedS = f"{changeS:.2f}"
percentage_change_formattedS = f"{percentage_changeS:.2f}%"

if not selected == "HELP":
    selected_company = st.selectbox("\n\nSearch for a Company", company)
    st.write("---")
    n = company.index(selected_company)
    selected_stocks = code[n]
    
    
    if selected == "HOME":
        
            col1,col2,col3=st.columns(3)
            with col1:
                st.subheader("Nifty")
                st.metric(label="", value=current_formattedN, delta=f"{change_formattedN} ({percentage_change_formattedN})")
            with col3:
                st.subheader("Sensex")
                st.metric(label="", value=current_formattedS, delta=f"{change_formattedS} ({percentage_change_formattedS})")
            st.write("---")      
       
        
    if selected_stocks:
        if n>0:
            st.subheader(f" {company[n]} ({code[n]})") 
            st.write("---")
        data = yf.download(tickers=selected_stocks, start="2015-01-01", end=date.today().strftime("%Y-%m-%d"))
        T = yf.Ticker(selected_stocks)
        if selected == "HOME":
            
            current = T.history(period='5d')['Close'].iloc[-1]
            last = T.history(period='5d')['Close'].iloc[-2]
            change = current - last
            percentage_change = (change / last) * 100
            current_formatted = f"₹{current:.2f}"
            change_formatted = f"{change:.2f}"
            percentage_change_formatted = f"{percentage_change:.2f}%"
            st.metric(label=T.info['longName'], value=current_formatted, delta=f"{change_formatted} ({percentage_change_formatted})")

            if change > 0:
                graphColour = 'green'
            else:
                graphColour = 'red'
            data_reset = data.reset_index()
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=data_reset['Date'], y=data_reset['Adj Close'], name="Prices", line=dict(color=graphColour)))
            fig.layout.update(title_text="Prices", xaxis_rangeslider_visible=True)
            st.plotly_chart(fig)
            st.write("---")
                
            close = T.info['previousClose']
            opening = T.info['open']
            h = T.info['dayHigh']
            l = T.info['dayLow']
            Fhigh = T.info['fiftyTwoWeekHigh']
            Flow = T.info['fiftyTwoWeekLow']
            v = T.info['volume']
            
            col1,col2=st.columns(2)
            with col1:
                st.write(f'**Opening Price** = ₹ {round(opening, 2)}')
                st.write(f'**Closing Price** = ₹ {round(close, 2)}')
            with col2:
                st.write(f'**Days Range** = ₹ {round(l, 2)} - ₹ {round(h, 2)}')
                st.write(f'**Volume** = {v}')
            st.write(f' **52 Week Low - High** = ₹ {round(Flow,2)} - ₹ {round(Fhigh,2)}')          
                
            today = datetime.today()
            start_date = today - timedelta(days=7)
            
            col1,col2=st.columns(2)
            data1 = yf.download(tickers=selected_stocks, start=start_date, end=today)
            
            data1.index = pd.to_datetime(data1.index).tz_localize('UTC').tz_convert('Asia/Kolkata')
                        
            data1.index = data1.index.strftime('%d-%m-%Y')
            rounded_data = {col: data1[col].round(2) for col in data1.columns if data1[col].dtype != 'object'}
            data_rounded = pd.DataFrame(rounded_data)
            st.write("---")
            st.subheader("Data Chart")
            st.dataframe(data_rounded)
        
        if selected == "LIVE PRICES":
            st.header("Live Price Updates")
            st.write("Updates only in Market Hours")
            col1,col2,col3=st.columns(3)
            with col1:
                live_button = st.button("Live Prices")

            if live_button:
                metric_placeholder = st.empty()
                with col2:
                    exit_button = st.button("Exit")
                exit_clicked = False 

                while not exit_clicked:
                    current = T.history(period='5d')['Close'].iloc[-1]
                    last = T.history(period='5d')['Close'].iloc[-2]
                    change = current - last
                    percentage_change = (change / last) * 100
                    current_formatted = f"₹{current:.2f}"
                    change_formatted = f"{change:.2f}"
                    percentage_change_formatted = f"{percentage_change:.2f}%"

                    metric_placeholder.subheader("Nifty 50")
                    metric_placeholder.metric(label="Live Prices", value=current_formatted, delta=f"{change_formatted} ({percentage_change_formatted})")

                    if exit_button:
                        exit_clicked = True  
                    time.sleep(0.25)
            st.write("---")
    
        if selected == "GRAPHS":
            
            st.header("GRAPHS")
            st.write("---")
            st.write("You can select time period of data ")
            col1,col2=st.columns(2)
            with col1:
                START = st.date_input('Enter start date', value=pd.to_datetime('2020-01-01'))
            with col2:
                END = st.date_input('Enter end date', value=pd.to_datetime(date.today().strftime("%Y-%m-%d")))
            
            d= yf.download(tickers=selected_stocks, start=START, end=END).reset_index()
            fig1 = go.Figure()
            fig1.add_trace(go.Scatter(x=d['Date'], y=d['Open'], name="Opening Price"))  
            fig1.add_trace(go.Scatter(x=d['Date'], y=d['Close'], name="Closing Price"))
            fig1.layout.update(title_text="Opening and Closing Prices", xaxis_rangeslider_visible=True)
            
            fig2 = go.Figure()
            fig2.add_trace(go.Scatter(x=d['Date'], y=d['High'], name="High"))  
            fig2.add_trace(go.Scatter(x=d['Date'], y=d['Low'], name="Low"))
            fig2.layout.update(title_text="Low and High Prices", xaxis_rangeslider_visible=True)
            
            fig3 = go.Figure()
            fig3.add_trace(go.Scatter(x=d['Date'], y=d['Volume'], name="Volume"))
            fig3.layout.update(title_text="Volume", xaxis_rangeslider_visible=True)
            
            st.write("Click to preview Graph")
            col4,col5,col6=st.columns(3)
            with col4:
               button3=st.button("Opening and Closing Prices")
            with col5:
               button4=st.button("Low and High")
            with col6:
                button5=st.button("Volume")
        
            if button3:
                st.write("Prices v/s Time")
                st.plotly_chart(fig1)  
                st.button("Exit  ")
            
            if button4:
                st.write("Prices v/s Time")
                
                st.plotly_chart(fig2)
                st.button("Exit     ")
            if button5:
                st.write("Volumes v/s Time")
                st.plotly_chart(fig3)
                st.button("Exit      ")
        
        if selected == "DATA":
            
            st.write("See Data in a Time Interval")
            col1, col2, col3 = st.columns(3)
            t = ["1m", "5m", "15m", "30m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"]
            
            today = datetime.today().date()
            start_date = (today - timedelta(days=7))
            
            with col3:
                selected_interval = st.selectbox("Select a Time Interval", t)
            with col1:
                start = st.date_input('Enter start date', value=start_date)
            with col2:
                end = st.date_input('Enter end date', value=today)
            
           
            
            if selected_interval == "1m":
                max_date_diff = timedelta(days=7)
            elif selected_interval in ["5m", "15m", "30m", "90m"]: 
                max_date_diff = timedelta(days=60)
            elif selected_interval in ["60m", "1h"]:
                max_date_diff = timedelta(days=730)
            else:
                max_date_diff = None
            if selected_interval == "5d":
                min_diff = timedelta(days=5)
            elif selected_interval == "1wk":
                min_diff = timedelta(days=7)
            elif selected_interval == "1mo":
                min_diff = timedelta(days=30)
            elif selected_interval == "3mo":
                min_diff = timedelta(days=90)
            else:
                min_diff = None
                    
            if max_date_diff and (end - start) > max_date_diff and selected_interval !="1m" :
                st.error(f"The selected time interval '{selected_interval}' allows a maximum date range of {max_date_diff.days} days.")
            elif min_diff and (end-start) < min_diff and selected_interval !="1m":
                st.error(f"The selected time interval '{selected_interval}' requires a minimum date range of {min_diff.days} days.")
            elif selected_interval == "1m" :
                if (end-start) > timedelta(days=7):
                    st.error(f"The selected time interval '{selected_interval}' allows a maximum date range of {timedelta(days=7).days} days.")
                else:
                    Data = T.history(start=start, end=end, interval=selected_interval)
                    try:
                        Data.index = Data.index.tz_localize(None)
                    except KeyError:
                        pass
                            
                    st.dataframe(Data)
                            
                    Data = Data.reset_index()
                    fig0 = go.Figure()
                    if 'Datetime' in Data.columns: 
                        fig0.add_trace(go.Scatter(x=Data['Datetime'], y=Data['Close'], mode='lines', name='Close'))
                    else:
                        fig0.add_trace(go.Scatter(x=Data['Date'], y=Data['Close'], mode='lines', name='Close'))
                    fig0.update_layout(title='Stock Prices',xaxis_rangeslider_visible=True)
                    st.plotly_chart(fig0)
            else:
                Data = T.history(start=start, end=end, interval=selected_interval)
                    
                try:
                    Data.index = Data.index.tz_localize(None)
                except KeyError:
                    pass
                
                        
                st.dataframe(Data)
                        
                Data = Data.reset_index()
                fig0 = go.Figure()
                if 'Datetime' in Data.columns: 
                    fig0.add_trace(go.Scatter(x=Data['Datetime'], y=Data['Close'], mode='lines', name='Close'))
                else:
                    fig0.add_trace(go.Scatter(x=Data['Date'], y=Data['Close'], mode='lines', name='Close'))
                fig0.update_layout(title='Stock Prices',xaxis_rangeslider_visible=True)
                st.plotly_chart(fig0)

            
        if selected == "TECHNICAL INDICATORS":
            
            st.header("Technical Indicators")
            data['SMA_50'] = data['Adj Close'].rolling(window=50).mean().round(2)
            data['SMA_200'] = data['Adj Close'].rolling(window=200).mean().round(2)
            
            def calculate_mfi(data, window=14):
                data['tp'] = (data['High'] + data['Low'] + data['Close']) / 3
                data['mf'] = data['tp'] * data['Volume']
                data['pmf'] = 0.0
                data['nmf'] = 0.0
                data.loc[data['tp'] > data['tp'].shift(1), 'pmf'] = data['mf']
                data.loc[data['tp'] < data['tp'].shift(1), 'nmf'] = data['mf']
                data['mfr'] = data['pmf'].rolling(window=window).sum() / data['nmf'].rolling(window=window).sum()
                data['mfi'] = 100 - (100 / (1 + data['mfr']))
                return data
            data_mfi = calculate_mfi(data)
            mfi_value = data_mfi['mfi'].iloc[-1]
            mfi_value = round(data_mfi['mfi'].iloc[-1], 2)
        
            data_reset = data.reset_index()
        
            fig1 = go.Figure()
            fig1.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], name="Prices"))
            fig1.add_trace(go.Scatter(x=data.index, y=data['SMA_50'], name="SMA 50"))
            fig1.add_trace(go.Scatter(x=data.index, y=data['SMA_200'], name="SMA 200"))
            fig1.update_layout(title="Stock Price with SMAs", xaxis_rangeslider_visible=True)
            
            fig_mfi = go.Figure()
            fig_mfi.add_trace(go.Scatter(x=data_mfi.index, y=data_mfi['mfi'], name="MFI"))
            fig_mfi.update_layout(title='Money Flow Index (MFI)',xaxis_title='Date',yaxis_title='MFI',xaxis_rangeslider_visible=True)
            
            col1,col2=st.columns(2)
            with col1:
                button1=st.button("Simple Moving Average")
            with col2:
                button2=st.button("Money Flow Index")
            if button1:
                st.plotly_chart(fig1)
                st.write("SMA-50 shows Short Term Average\n\n SMA-200 shows Long Term Average\n\n If in the graph:-\n\n  SMA-50 is above SMA-200 : Bullish Trend\n\n Price above of both SMA-50 and SMA-200 : Bullish Trend\n\n SMA-50 below SMA-200 : Bearish Trend\n\n Prices below of both SMA-50 and SMA-200 : Bearish Trend")
                st.button("Exit   ")
                
            if button2:
                st.metric(label="MFI", value=mfi_value) 
                st.plotly_chart(fig_mfi)
                st.write("MFI Index Value ranges from 0 - 100.\n\n MFI >= 80 means the stock is overbought and might face a downward correction.\n\n MFi <= 20 means the stock is oversold and might face a upward correction\n\n MFI between 20 and 80 means the stock is balanced.")
                st.button("Exit     ")
            
        if selected == "INFORMATION":
            st.header("Information")
            
            business_summary = T.info.get('longBusinessSummary')
            if business_summary:
                st.subheader("Business Summary")
                st.caption(business_summary)
                st.write("---")
            
            industry = T.info.get('industry')
            if industry:
                st.write(f"**Industry:** {industry}")
            
            sector = T.info.get('sector')
            if sector:
                st.write(f"Sector: {sector}")
                
            cap = T.info.get('marketCap')
            if industry:
                st.write(f"**Market Capitalisation**: ₹ {cap}")
                
            div = T.info.get('dividendRate')
            if industry:
                st.write(f"**Dividend Rate:** {div:.2f}")
            
            pe = T.info.get('trailingPE')
            if industry:
                st.write(f"**P/E Ratio:** {pe:.2f}")
            
            total_cash = T.info.get('totalCash')
            if total_cash:
                st.write(f"**Total Cash**: ₹ {total_cash}")
            
            total_debt = T.info.get('totalDebt')
            if total_debt:
                st.write(f"**Debt Amount**: ₹ {total_debt}")
            
            total_revenue = T.info.get('totalRevenue')
            if total_revenue:
                st.write(f"Total Revenue**: ₹ {total_revenue}")
            
            free_cash_flow = T.info.get('freeCashflow')
            if free_cash_flow:
                st.write(f"Free Cash Flow**: ₹ {free_cash_flow}")

            
            st.write("---")
            st.subheader("News")
            news = T.news
            if news:
                for item in news:
                    title = item['title']
                    source = item['publisher']
                    url = item['link']
                    
                    st.markdown(f"Title: {title} -- <a href='{url}' style='color: lightgrey;'>See More</a>", unsafe_allow_html=True)
                    st.caption(f"**Source:** {source}")
                   
          
        if selected == "FORECAST":
            st.header("Forecasting")
            st.write("---")
            n_years = st.slider("Months Of Prediction: ", 1, 24)
            period = n_years * 30
            
            # Preprocess the data
            df = data.reset_index()[["Date", "Close"]].copy()
            df["Date"] = pd.to_datetime(df["Date"])
            df["DayOfWeek"] = df["Date"].dt.dayofweek
            df = df[(df["DayOfWeek"] != 5) & (df["DayOfWeek"] != 6)]
            df = df.rename(columns={"Date": "ds", "Close": "y"})
            
            # Train the Prophet model
            m = Prophet(daily_seasonality=True, yearly_seasonality=True)
            m.fit(df)
            
            # Make future dataframe and predictions
            future = m.make_future_dataframe(periods=period)
            forecast = m.predict(future)
            forecast1 = forecast.rename(columns={'ds':'Date','yhat':'Predicted Prices','yhat_lower':'Predicted Lowest','yhat_upper':'Predicted Highest'})
            forelist = ['Date', 'Predicted Prices', 'Predicted Lowest', 'Predicted Highest']
            
            st.write("---")
            st.subheader("Forecast Data")
            st.write(forecast1[forelist])
            st.write('---')
            
            # Prediction interval
            st.subheader("Prediction in Interval of Time")
            Start = st.date_input('Enter start date', value=None)
            End = st.date_input('Enter end date', value=None)
            if Start != End:
                selected_forecast = forecast1.loc[(forecast1['Date'] > pd.to_datetime(Start)) & (forecast1['Date'] <= pd.to_datetime(End))]
                st.write(selected_forecast[forelist])
            st.write("---")
            
            # Forecasted Data Graphs
            st.subheader('Forecasted Data Graphs')
            st.write("Actual Prices v/s Predicted Prices")
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predicted Prices', line=dict(color='red')))
            fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines', name='Actual Prices', marker=dict(color='blue')))
            fig.update_layout(xaxis_rangeslider_visible=False)
            st.plotly_chart(fig)
            
            st.write("Forecast Components")
            fig2 = m.plot_components(forecast)
            st.write(fig2)
            
            st.button("Exit")
            st.write("---")
        
if selected=="HELP":
    st.write("---")
    st.header("Help")
    st.write("This Web application allow users to visualize and analyze the stock prices of different companies and predict their future performance based on various factors.\nUser can see various graphs and future predicted prices.\n\nThis a useful tool for investors and traders to make informed decisions about their investments and gain insights into the stock market trends.")
    st.write("Here you can see how to use stock analysis and prediction.\nSearch for a comapny and you can see it's current stats.\nMenu on the left can be used for various features. User can select to see GRAPHS, DATA, INFORMATION, TECHNICAL INDICATORS & FORECAST.\nUser can use given buttons options to get more information.")
    st.caption("Contact us :-\n\nIn case of any inconvienience or any issue, Report at Email : - stocks_analysis@gmail.com")
        

In [7]:
import streamlit as st

# Define companies and their corresponding stock codes
companies = {
    "Apple Inc (AAPL)": "AAPL",
    "Microsoft Corp (MSFT)": "MSFT",
    "Google LLC (GOOGL)": "GOOGL"
}

# App title
st.title("Stock Selector App")

# Sidebar for navigation
selected = st.sidebar.radio("Navigate", ["HOME", "STOCKS"])

if selected == "HOME":
    st.subheader("Welcome to the Stock Selector App!")
    st.write("Use this app to select a company and fetch its stock ticker.")
    st.write("---")

elif selected == "STOCKS":
    st.subheader("Stock Ticker Lookup")

    # Dropdown to select a company
    selected_company = st.selectbox("Select a company:", list(companies.keys()))
    
    # Retrieve the stock ticker for the selected company
    selected_stocks = companies[selected_company]
    
    # Display the selected stock ticker
    st.write(f"**Selected Company:** {selected_company}")
    st.write(f"**Stock Ticker:** {selected_stocks}")


2024-12-03 10:15:13.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-03 10:15:13.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-03 10:15:13.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-03 10:15:13.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-03 10:15:13.979 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-03 10:15:13.981 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-03 10:15:13.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-03 10:15:13.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar